## MediPal -- Evaluation -- RAGAS

### In this section, I evaluate MediPal by RAGAS. I keep everything the same but change different set of models to see the differents. 

##### I set exact the same evaluation dataset, matrics and environment, comparing two sets of models. They have almost the same size but one set is general models, another one is fine-tuned on medical documents.

| Medical Team      |  General Team                      |
| ----------------- | ---------------------------------- |
| sentence-transformers/embeddinggemma-300m-medical | google/embeddinggemma-300m |
| ncbi/MedCPT-Cross-Encoder | cross-encoder/ms-marco-MiniLM-L6-v2 |
| ContactDoctor/Bio-Medical-Llama-3-8B | meta-llama/Meta-Llama-3-8B-Instruct |


##### Matrics: answer_relevancy, faithfulness, context_precision, context_recall, answer_correctness 

In [ ]:
from src.medipal import medipal_app
from src.settings import AgentState
from datasets import Dataset
import pandas as pd
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision, context_recall, answer_correctness

In [ ]:
def print_metrics_table(metrics: dict):
    # Convert dict to DataFrame with two columns
    df = pd.DataFrame(metrics.items(), columns=["Metric", "Score"])
    # Round numbers to 4 decimal places
    df["Score"] = df["Score"].round(4)
    display(df)

# Example input
metrics = {
    'answer_relevancy': 0.9281,
    'faithfulness': 0.8235,
    'context_precision': 0.8824,
    'context_recall': 0.8235,
    'answer_correctness': 0.7584
}

print_metrics_table(metrics)

In [ ]:
qa_pairs = [
    {
        "question": "What conditions can phenylephrine help relieve?",
        "answer": "Phenylephrine can relieve nasal discomfort caused by colds, allergies, hay fever, and also help with sinus congestion and pressure."
    },
    {
        "question": "Does phenylephrine cure the cause of symptoms?",
        "answer": "No. Phenylephrine only relieves the symptoms but does not treat the underlying cause or speed up recovery."
    },
    {
        "question": "How does phenylephrine work in the body?",
        "answer": "Phenylephrine is a nasal decongestant. It works by reducing swelling of the blood vessels in the nasal passages."
    },
	{
        "question": "What allergies should you tell your doctor or pharmacist about before using pyrethrin and piperonyl butoxide shampoo?",
        "answer": "You should mention if you are allergic to pyrethrin, piperonyl butoxide, chrysanthemums, ragweed, any other medications, or any of the shampoo's ingredients."
    },
    {
        "question": "Why is it important to tell your doctor about other medications or supplements before using pyrethrin and piperonyl butoxide shampoo?",
        "answer": "Because prescription and nonprescription medications, vitamins, nutritional supplements, and herbal products may interact with the shampoo, so your doctor needs to know what you are taking or plan to take."
    },
    {
        "question": "What should you tell your doctor regarding your health history before using pyrethrin and piperonyl butoxide shampoo?",
        "answer": "You should tell your doctor if you have or have ever had skin conditions, or if you are pregnant, planning to become pregnant, or are breast-feeding."
    },
	{
        "question": "What should you do if someone swallows pyrethrin and piperonyl butoxide shampoo?",
        "answer": "Call your local poison control center immediately."
    },
    {
        "question": "What should you do if the person has collapsed or is not breathing after swallowing pyrethrin and piperonyl butoxide shampoo?",
        "answer": "Call local emergency services at 911 right away."
    },
    {
        "question": "Who should you contact first if pyrethrin and piperonyl butoxide shampoo is swallowed?",
        "answer": "You should first contact your local poison control center for instructions."
    },
	{
        "question": "What conditions can antipyrine and benzocaine otic help relieve?",
        "answer": "It can relieve ear pain and swelling caused by middle ear infections."
    },
    {
        "question": "Can antipyrine and benzocaine otic be used with antibiotics?",
        "answer": "Yes. It may be used along with antibiotics to treat an ear infection."
    },
    {
        "question": "How does the combination of antipyrine and benzocaine work?",
        "answer": "They are analgesics that work by reducing pain and discomfort in the ear."
    },
	{
        "question": "What is methylprednisolone injection used to treat?",
        "answer": "It is used to treat severe allergic reactions, multiple sclerosis, lupus, gastrointestinal disease, certain types of arthritis, and conditions affecting the blood, skin, eyes, nervous system, thyroid, kidneys, and lungs."
    },
    {
        "question": "Can methylprednisolone injection be used with other medications?",
        "answer": "Yes. It is sometimes used in combination with other medications to treat symptoms of low corticosteroid levels."
    },
    {
        "question": "What class of medications does methylprednisolone injection belong to?",
        "answer": "It belongs to a class of medications called corticosteroids."
    },
    {
        "question": "How does methylprednisolone injection help people with low corticosteroid levels?",
        "answer": "It replaces steroids that are normally produced naturally by the body."
    },
    {
        "question": "How does methylprednisolone injection treat other conditions?",
        "answer": "It works by reducing swelling and redness and by changing the way the immune system functions."
    }	
]

In [ ]:
questions = []
model_answers = []
retrieved_contexts_lists: list[list] = []
reference_answers = []
for i, s in enumerate(qa_pairs):
    question = s["question"]
    answer = s["answer"]
    questions.append(question)
    reference_answers.append(answer)    
    query = AgentState(query=question, session_id=1,wiki_used=False,brave_used=False,rewrite_counter=0,regenerate_counter=0)
    result = medipal_app.invoke(query) 
    retrieved_contexts_lists.append([result["retrieved_doc"]])
    model_answers.append(result["generation"])   


data = Dataset.from_dict({
  "question": questions,                 # list[str]
  "answer": model_answers,               # list[str]
  "contexts": retrieved_contexts_lists,  # list[list[str]]
  "ground_truth": reference_answers      # list[str] (optional but needed for correctness)
})
# Everything is ready for evaluation
result = evaluate(
  data,
  metrics=[answer_relevancy, faithfulness, context_precision, context_recall, answer_correctness]
)

logging_print(result, print_out=True)   